# YOLO
The goal of this notebook is to do transfer learning of Fishial´s already implemented YOLO model. We do this by using Ultralytics

In [2]:
from ultralytics import YOLO
import os
import requests
import logging
from zipfile import ZipFile
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
import copy
from IPython.display import Image, display

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [3]:
DATA_DIR = "../data_csv/"
timor_leste_data_path = os.path.join(DATA_DIR, "timor-leste.csv") # Annotation info for ground truth
images_path = "../data_images/"

In [4]:
# Filter out relevant images
relevant_species = ['Alectis ciliaris', 'Aphareus rutilans', 'Caranx ignobilis', 'Caranx lugubris', 'Caranx melampygus', 'Caranx sexfasciatus', 'Chirocentrus dorab', 'Chirocentrus nudus', 'Decapterus macrosoma', 'Elagatis bipinnulata', 'Epinephelus maculatus', 'Epinephelus radiatus', 'Etelis carbunculus', 'Gymnocranius grandoculis', 'Katsuwonus pelamis', 'Lethrinus atkinsoni', 'Lethrinus erythracanthus', 'Lethrinus obsoletus', 'Lethrinus ornatus', 'Lutjanus bohar', 'Lutjanus fulviflamma', 'Lutjanus fulvus', 'Lutjanus gibbus', 'Lutjanus johnii', 'Lutjanus kasmira', 'Lutjanus rivulatus', 'Lutjanus russellii', 'Lutjanus timoriensis', 'Monotaxis grandoculis', 'Psettodes erumei', 'Rastrelliger kanagurta', 'Sardinella albella', 'Scolopsis lineata', 'Scolopsis vosmeri', 'Scomberoides lysan', 'Scomberomorus commerson', 'Seriola dumerili', 'Variola albimarginata']

# Read annotation info (Timor-leste)
df_tl_ann = pd.read_csv(timor_leste_data_path, encoding="utf-8-sig", header=0, skiprows=1)
df_tl_ann = df_tl_ann[["image_file", "catch_name_en", "Species_name", "Family"]]

# Filter by species in relevant_species
df_filtered = df_tl_ann[df_tl_ann["Species_name"].isin(relevant_species)]

# Keep only images that actually exist
existing_files = set(os.listdir(images_path))

df_filtered = df_filtered[df_filtered["image_file"].isin(existing_files)]

# Convert to list of filenames
filtered_images= df_filtered["image_file"].tolist()

print("Total annotated images:", len(df_tl_ann))
print("Relevant images", len(df_filtered))
print("Unique final JPGs:", len(set(filtered_images)))


Total annotated images: 603
Relevant images 248
Unique final JPGs: 217


In [14]:
df_filtered.to_csv("filtered_timor_leste_annotations.csv", index=False)

In [19]:
# Load a pretrained YOLO11n model
model = YOLO("yolo11n.pt")

# Train the model
train_results = model.train(
    data="dataset/data.yaml",
    epochs=1,
    imgsz=640,
    device="cpu",
    patience=50
)

# Evaluate the model's performance on the validation set
metrics = model.val()

New https://pypi.org/project/ultralytics/8.3.235 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.234  Python-3.12.12 torch-2.9.1+cpu CPU (12th Gen Intel Core i5-12500)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimiz

In [ ]:
display(Image(filename="runs/classify/train/results.png"))
display(Image(filename="runs/classify/val/confusion_matrix.png"))

# Preditcion

In [6]:
import os
import csv
from ultralytics import YOLO

model = YOLO("best.pt")  # your detection model

# Paths to images
image_paths = [os.path.join("../data_images/", f) for f in filtered_images]

results = model.predict(
    source=image_paths,
    imgsz=480,
)

# -------------------------------------------------------
# Create CSV file
# -------------------------------------------------------
csv_path = "detection_results.csv"

with open(csv_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["image", "class", "confidence", "x1", "y1", "x2", "y2"])

    # Iterate through model predictions
    for r in results:
        boxes = getattr(r, "boxes", None)
        image_name = os.path.basename(r.path)

        # No detections
        if boxes is None or len(boxes) == 0:
            writer.writerow([image_name, "no detections", "", "", "", "", ""])
            continue

        # Extract detections
        xyxy = boxes.xyxy.cpu().numpy()
        scores = boxes.conf.cpu().numpy()
        classes = boxes.cls.cpu().numpy()

        for (x1, y1, x2, y2), conf, cls in zip(xyxy, scores, classes):
            class_name = model.names[int(cls)]
            writer.writerow([
                image_name,
                class_name,
                f"{conf:.4f}",
                int(x1), int(y1), int(x2), int(y2)
            ])

print(f"\nCSV file saved to: {csv_path}")




0: 480x480 (no detections), 45.9ms
1: 480x480 (no detections), 45.9ms
2: 480x480 (no detections), 45.9ms
3: 480x480 1 evistias_acutirostris, 45.9ms
4: 480x480 (no detections), 45.9ms
5: 480x480 (no detections), 45.9ms
6: 480x480 (no detections), 45.9ms
7: 480x480 (no detections), 45.9ms
8: 480x480 (no detections), 45.9ms
9: 480x480 (no detections), 45.9ms
10: 480x480 1 epinephelus_ongus, 45.9ms
11: 480x480 (no detections), 45.9ms
12: 480x480 (no detections), 45.9ms
13: 480x480 (no detections), 45.9ms
14: 480x480 (no detections), 45.9ms
15: 480x480 (no detections), 45.9ms
16: 480x480 (no detections), 45.9ms
17: 480x480 (no detections), 45.9ms
18: 480x480 (no detections), 45.9ms
19: 480x480 (no detections), 45.9ms
20: 480x480 (no detections), 45.9ms
21: 480x480 (no detections), 45.9ms
22: 480x480 (no detections), 45.9ms
23: 480x480 (no detections), 45.9ms
24: 480x480 (no detections), 45.9ms
25: 480x480 (no detections), 45.9ms
26: 480x480 (no detections), 45.9ms
27: 480x480 1 evistias_ac

In [11]:
import os
import pandas as pd
from ultralytics import YOLO

# ---------- config ----------
MODEL_PATH = "best.pt"
IMAGE_PATHS = [os.path.join("../data_images/", f) for f in filtered_images]  # filtered_images must exist
CSV_OUT = "predictions_with_groundtruth_matches.csv"
GT_FILENAME_COL = "image_file"          # column in df_tl_ann to join on
GT_SPECIES_COL = "Species_name"       # ground truth species column name in df_tl_ann
# ----------------------------

# Load model
model = YOLO(MODEL_PATH)

# Run predictions (you can tune imgsz, conf, etc.)
results = model.predict(source=IMAGE_PATHS, imgsz=480)

# Build predictions dataframe
pred_rows = []
for r in results:
    image_name = os.path.basename(r.path)
    boxes = getattr(r, "boxes", None)

    if boxes is None or len(boxes) == 0:
        pred_rows.append({
            "image_file": image_name,
            "pred_class": None,
            "pred_conf": None,
            "pred_x1": None,
            "pred_y1": None,
            "pred_x2": None,
            "pred_y2": None,
        })
        continue

    xyxy = boxes.xyxy.cpu().numpy()
    scores = boxes.conf.cpu().numpy()
    classes = boxes.cls.cpu().numpy()

    for (x1, y1, x2, y2), conf, cls in zip(xyxy, scores, classes):
        pred_rows.append({
            "image_file": image_name,
            "pred_class": model.names[int(cls)] if cls is not None else None,
            "pred_conf": float(conf),
            "pred_x1": int(x1),
            "pred_y1": int(y1),
            "pred_x2": int(x2),
            "pred_y2": int(y2),
        })

df_preds = pd.DataFrame(pred_rows)

# ---------- load ground truth ----------
# If df_tl_ann is not already loaded, load it here, e.g.:
# df_tl_ann = pd.read_csv("timor_leste_annotations.csv")
# Ensure it contains the expected columns
if GT_FILENAME_COL not in df_tl_ann.columns:
    raise KeyError(f"Ground truth dataframe must contain column '{GT_FILENAME_COL}'")
if GT_SPECIES_COL not in df_tl_ann.columns:
    raise KeyError(f"Ground truth dataframe must contain species column '{GT_SPECIES_COL}'")

# If ground truth has multiple rows per filename (multiple labeled fish per image),
# aggregate species into a list per filename to allow matching any of them.
df_gt_grouped = (
    df_tl_ann
    .groupby(GT_FILENAME_COL)[GT_SPECIES_COL]
    .apply(lambda s: list(s.dropna().astype(str)))
    .reset_index()
    .rename(columns={GT_SPECIES_COL: "gt_species_list"})
)

# Merge predictions with grouped ground truth (left join keeps predictions even if no GT)
df_merged = df_preds.merge(df_gt_grouped, on="image_file", how="left")

# ---------- helper for normalization ----------
def normalize_name(s):
    if pd.isna(s) or s is None:
        return ""
    # lowercase, strip whitespace, replace underscores, collapse multiple spaces
    s2 = str(s).lower().strip().replace("_", " ")
    s2 = " ".join(s2.split())
    return s2

# Normalize GT species lists & prediction class for comparison
df_merged["pred_class_norm"] = df_merged["pred_class"].apply(normalize_name)
df_merged["gt_species_norm_list"] = df_merged["gt_species_list"].apply(
    lambda lst: [normalize_name(x) for x in lst] if isinstance(lst, list) else []
)

# ---------- compute match column ----------
def compare_row(row):
    pred = row["pred_class_norm"]
    gt_list = row["gt_species_norm_list"]

    if (pred == "" or pred is None) and (not gt_list):
        # no detection and no gt
        return "NO_DETECTION_AND_NO_GT"
    if pred == "" or pred is None:
        # no detection but GT exists
        return "NO_DETECTION"
    if not gt_list:
        # prediction exists but no ground truth
        return "NO_GT"
    # if any exact match in normalized strings:
    if pred in gt_list:
        return "MATCH"
    # optionally: fuzzy matching can be done here (e.g., substring/in operator)
    # try substring match (pred contained in any gt or vice versa)
    for g in gt_list:
        if pred in g or g in pred:
            return "MATCH_SUBSTRING"
    return "MISMATCH"

df_merged["match"] = df_merged.apply(compare_row, axis=1)

# ---------- optional: summary stats ----------
summary = df_merged["match"].value_counts(dropna=False)
print("Match summary:\n", summary)

# Save to CSV
df_merged.to_csv(CSV_OUT, index=False)
print(f"\nSaved merged file with matches to: {CSV_OUT}")



0: 480x480 (no detections), 48.5ms
1: 480x480 (no detections), 48.5ms
2: 480x480 (no detections), 48.5ms
3: 480x480 1 evistias_acutirostris, 48.5ms
4: 480x480 (no detections), 48.5ms
5: 480x480 (no detections), 48.5ms
6: 480x480 (no detections), 48.5ms
7: 480x480 (no detections), 48.5ms
8: 480x480 (no detections), 48.5ms
9: 480x480 (no detections), 48.5ms
10: 480x480 1 epinephelus_ongus, 48.5ms
11: 480x480 (no detections), 48.5ms
12: 480x480 (no detections), 48.5ms
13: 480x480 (no detections), 48.5ms
14: 480x480 (no detections), 48.5ms
15: 480x480 (no detections), 48.5ms
16: 480x480 (no detections), 48.5ms
17: 480x480 (no detections), 48.5ms
18: 480x480 (no detections), 48.5ms
19: 480x480 (no detections), 48.5ms
20: 480x480 (no detections), 48.5ms
21: 480x480 (no detections), 48.5ms
22: 480x480 (no detections), 48.5ms
23: 480x480 (no detections), 48.5ms
24: 480x480 (no detections), 48.5ms
25: 480x480 (no detections), 48.5ms
26: 480x480 (no detections), 48.5ms
27: 480x480 1 evistias_ac

In [12]:
# Count total rows with predictions (pred_class not None)
total_predictions = df_merged[~df_merged["pred_class"].isna()].shape[0]

# Count MATCH
match_count = (df_merged["match"] == "MATCH").sum()

# Optional: also include substring matches
match_incl_substring = df_merged["match"].isin(["MATCH", "MATCH_SUBSTRING"]).sum()

print("---- MATCH STATISTICS ----")
print(f"Total predictions: {total_predictions}")
print(f"Exact MATCH: {match_count}")
print(f"Exact MATCH percentage: {match_count / total_predictions * 100:.2f}%")

print("\nIncluding MATCH_SUBSTRING:")
print(f"Total MATCH (incl. substring): {match_incl_substring}")
print(f"MATCH percentage (incl. substring): {match_incl_substring / total_predictions * 100:.2f}%")


---- MATCH STATISTICS ----
Total predictions: 89
Exact MATCH: 1
Exact MATCH percentage: 1.12%

Including MATCH_SUBSTRING:
Total MATCH (incl. substring): 1
MATCH percentage (incl. substring): 1.12%
